# XAS Workflow Task 1

This notebook contains the first task of the XAS processing workflow. 

The break up of the task consist of the following steps 

| Task                            | Input                                         | Output
| -------------                   |-------------                                  | -----  
| Create Athena project      |                                               | 
| 1. Import data                |File: fes2_rt01_mar02.xmu                      | 
| 2. Normalisation              |Parameters: Pre-edge range = -117.00 to 30.000 |
| 3. Save Athena Project        |                                               |File: FeS2_01.prj

For more details about larch, see https://xraypy.github.io/xraylarch/xafs/


In [ ]:
from larch.io import read_ascii, merge_groups
from pathlib import Path

from larch.symboltable import Group

#reading all with the same extension files from a dir
def get_files_list(source_dir, f_pattern):
    i_counter = 0
    source_dir = Path(source_dir)
    files_list = []
    for filepath in sorted(source_dir.glob(f_pattern)):
        i_counter += 1
        files_list.append(filepath)
    return files_list


# From BR demo at: https://vimeo.com/340207346 07:01
#file_path = "D:/PSDI/PSDI_test/psdi_data/pub_037/XAFS_prj/Sn K-edge/ascii/" # local
#file_path = "C:/harwell/PSDI_test/wf_data/pub_037/XAFS_prj/Sn K-edge/ascii/" # pc

file_path = "./"
#file_name_pattern = "26????_PtSn_OCH_Abu*"
#file_name_pattern = "26????_PtSn_OCA_H2_Abu*"

file_name_pattern = "*xmu"

file_names = get_files_list(file_path,file_name_pattern)

#labels_sn_k = 'energy time i0 it iref mu lnitiref'
labels_fes2 = "energy mu i0"

use_labels = labels_fes2
#file_names = ['fes2_rt01_mar02.xmu']

groups_list=[]

# read ascii files
# labels tell the reader how to name the resulting columns
# eneregy and mu are the most used labels

# if merge read in group and then merge
for a_file in file_names:
    a_group = read_ascii(a_file, labels=use_labels)
    groups_list.append(a_group)

# merge files
merged_xas = merge_groups (groups_list)

# using vars(fe_xas) we see that the object has the following properties: 
# path, filename, header, data, attrs, energy, xmu, i0
# vars(fe_xas)

In [ ]:
get_files_list(Path(file_path),"26????_PtSn_OCH_Abu*")

In [ ]:
# the following show the contents of each
for fe_xas in groups_list:
    print("path:\t\t", fe_xas.path)
    print("filename:\t", fe_xas.filename)
    print(fe_xas.header)
    print(fe_xas.data)
    print(fe_xas.attrs)
    print(fe_xas.array_labels)
    print('energy:', fe_xas.energy)
    print('i0:',fe_xas.i0)
    print('xmu:',fe_xas.mu)


## Calculate and plot $\mu$
Knowing the elements of the input data set it is possible to start manipulating and visualising the XAFS data. 

In this case $\mu$ is provided, so there is no need to calculate it, just rename it as mu (required by Larch for processing) and plot it agains the energy.


In [ ]:
from numpy import log
from larch.utils import group2dict, dict2group

# Calculate mu 
def get_mu(xafs_group):
    # calculate mu
    mu_e = xafs_group.mu
    # get a dictionary from te group
    xafs_dict = group2dict(xafs_group)
    # add mu to the dictionary
    xafs_dict['mu'] = mu_e
    xafs_group = dict2group(xafs_dict)
    return xafs_group

for fe_xas in groups_list:
    fe_xas = get_mu(fe_xas)

    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.plot(fe_xas.energy, fe_xas.mu, label='$\mu$(x)' + fe_xas.filename )
    plt.xlabel('Energy')
    plt.ylabel('$\mu$')
    plt.legend() # include the leyend in the plot
    plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid


    

In [ ]:
merged_xas = get_mu(merged_xas)
plt.plot(merged_xas.energy, merged_xas.mu, label='$\mu$(x)' + 'merge' )
plt.xlabel('Energy')
plt.ylabel('$\mu$')
plt.legend() # include the leyend in the plot
plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid

In [ ]:
# https://vimeo.com/340207346 25:00
from larch.xafs import pre_edge
#calculate pre-edge and post edge and add them to group
# as with the basic workflow we want to avoid the data below E0 - 117.00

for fe_xas in groups_list:
    pre_edge(fe_xas, group=fe_xas, pre1=-150, pre2=-60)

    plt.plot(fe_xas.energy, fe_xas.pre_edge, 'g', label='pre-edge') # plot pre-edge in green
    plt.plot(fe_xas.energy, fe_xas.post_edge, 'r', label='post-edge')# plot post-edge in green
    plt.plot(fe_xas.energy, fe_xas.mu, 'b', label=fe_xas.filename) # plot mu in blue
    plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
    plt.xlabel('Energy (eV)') # label y graph
    plt.ylabel('x$\mu$(E)') # label y axis
    plt.title("pre-edge and post_edge fitting to $\mu$")
    plt.legend() # show legend

In [ ]:
pre_edge(merged_xas, group=merged_xas, pre1=-150, pre2=-60)

plt.plot(merged_xas.energy, merged_xas.pre_edge, 'g', label='pre-edge') # plot pre-edge in green
plt.plot(merged_xas.energy, merged_xas.post_edge, 'r', label='post-edge')# plot post-edge in green
plt.plot(merged_xas.energy, merged_xas.mu, 'b', label="merge") # plot mu in blue
plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
plt.xlabel('Energy (eV)') # label y graph
plt.ylabel('x$\mu$(E)') # label y axis
plt.title("pre-edge and post_edge fitting to $\mu$")
plt.legend() # show legend

In [ ]:
# https://vimeo.com/340207346 27:00
for fe_xas in groups_list:
    plt.plot(fe_xas.energy, fe_xas.flat, label=fe_xas.filename) # plot flattened and normalised energy
    plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
    plt.xlabel('Energy (eV)') # label y graph
    plt.ylabel(r'normalised x$\mu$(E)') # label y axis
    plt.legend() # show legend


In [ ]:

plt.plot(merged_xas.energy, merged_xas.flat, label='merge') # plot flattened and normalised energy
plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
plt.xlabel('Energy (eV)') # label y graph
plt.ylabel(r'normalised x$\mu$(E)') # label y axis
plt.legend() # show legend


In [ ]:
# import the larch.io libraries for managing athena files
from larch.io import create_athena, read_athena, extract_athenagroup

# save as an athena project

project_name = 'PtSn_OCA_H2_Abu.prj'#'PtSn_OCH_Abu.prj' #'fes2_larch.prj'
fe_project = create_athena(project_name)
for fe_xas in groups_list:
    fe_project.add_group(fe_xas)
merged_xas.filename = "new_name"
merged_xas.path = "new_path"
fe_project.add_group(merged_xas)
fe_project.group_names[4]="merged group"
fe_project.save()
vars(fe_project)


In [ ]:
dir(merged_xas)